In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time

torch.manual_seed(42)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset_train = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
dataset_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

train_loader = DataLoader(dataset_train, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=64, shuffle=False)


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size=784, hidden_sizes=[128, 64], output_size=10, activation_fn=nn.ReLU):
        super(MLP, self).__init__()
        layers = []
        layers.append(nn.Linear(input_size, hidden_sizes[0]))
        layers.append(activation_fn())
        for i in range(len(hidden_sizes) - 1):
            layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
            layers.append(activation_fn())
        layers.append(nn.Linear(hidden_sizes[-1], output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.model(x)

def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoca [{epoch + 1}/{epochs}], Perdida: {running_loss / len(train_loader):.4f}")

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f"Exactitud: {accuracy * 100:.2f}%")
    return accuracy

experimentos = [
    {"hidden_sizes": [128, 64], "activation_fn": nn.ReLU, "lr": 0.01, "batch_size": 64, "epochs": 10},
    {"hidden_sizes": [256, 128], "activation_fn": nn.Tanh, "lr": 0.001, "batch_size": 32, "epochs": 10},
    {"hidden_sizes": [512, 256, 128], "activation_fn": nn.ReLU, "lr": 0.005, "batch_size": 128, "epochs": 15},
]

resultados = []

for i, config in enumerate(experimentos):
    print(f"\Experimento {i + 1}")
    model = MLP(hidden_sizes=config["hidden_sizes"], activation_fn=config["activation_fn"])
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])

    train_loader = DataLoader(dataset_train, batch_size=config["batch_size"], shuffle=True)

    train_model(model, train_loader, criterion, optimizer, epochs=config["epochs"])
    accuracy = evaluate_model(model, test_loader)
    resultados.append({"config": config, "accuracy": accuracy})

print("\nResumen de experimentos")
for i, resultado in enumerate(resultados):
    print(f"Experimento {i + 1}: Exactitud = {resultado['accuracy'] * 100:.2f}%, Config = {resultado['config']}")

mejor_experimento = max(resultados, key=lambda x: x["accuracy"])
print(f"\nMejor Modelo: Exactitud = {mejor_experimento['accuracy'] * 100:.2f}%, Config = {mejor_experimento['config']}")

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_filters=[32, 64], fc_neurons=128, dropout=0.5):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, num_filters[0], kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(num_filters[0], num_filters[1], kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(num_filters[1] * 7 * 7, fc_neurons),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(fc_neurons, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

def train_model(model, train_loader, criterion, optimizer, epocas=10):
    model.train()
    for epoca in range(epocas):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoca [{epoca + 1}/{epocas}], Perdida: {running_loss / len(train_loader):.4f}")

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f"Exactitud: {accuracy * 100:.2f}%")
    return accuracy

experiments = [
    {"num_filters": [32, 64], "fc_neurons": 128, "dropout": 0.5, "lr": 0.01, "epocas": 10},
    {"num_filters": [64, 128], "fc_neurons": 256, "dropout": 0.3, "lr": 0.001, "epocas": 10},
    {"num_filters": [16, 32], "fc_neurons": 64, "dropout": 0.2, "lr": 0.005, "epocas": 15},
]

results = []

for i, config in enumerate(experiments):
    print(f"\nExperimento {i + 1}")
    model = CNN(num_filters=config["num_filters"], fc_neurons=config["fc_neurons"], dropout=config["dropout"])
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])
    
    start_time = time.time()  # Usando time después de importarlo
    train_model(model, train_loader, criterion, optimizer, epocas=config["epocas"])
    elapsed_time = time.time() - start_time  # Tiempo transcurrido
    accuracy = evaluate_model(model, test_loader)
    results.append({"config": config, "accuracy": accuracy, "time": elapsed_time})

# Reporte final
print("\nSuma de Experimentos")
for i, result in enumerate(results):
    print(f"Experimiento {i + 1}: Exactitud = {result['accuracy'] * 100:.2f}%, Tiempo = {result['time']:.2f}s, Config = {result['config']}")

# Selección del mejor modelo
best_experiment = max(results, key=lambda x: x["accuracy"])
print(f"\nMejor Modelo: Exactitud = {best_experiment['accuracy'] * 100:.2f}%, Config = {best_experiment['config']}")

# Comparación final con el MLP
print("\nComparacion con MLP")
mlp_time = 15.0  # Tiempo estimado del MLP, ajustar según los resultados
mlp_accuracy = 0.95  # Accuracy estimado del MLP, ajustar según los resultados

print(f"MLP - Exactitud: {mlp_accuracy * 100:.2f}%, Time: {mlp_time:.2f}s")
print(f"Mejor CNN - Exactitud: {best_experiment['accuracy'] * 100:.2f}%, Tiempo: {best_experiment['time']:.2f}s")